In [1]:
# !pip install timm einops
# !wget https://github.com/Dao-AILab/flash-attention/releases/download/v2.6.3/flash_attn-2.6.3+cu123torch2.4cxx11abiFALSE-cp310-cp310-linux_x86_64.whl
# !pip install --no-dependencies --upgrade flash_attn-2.6.3+cu123torch2.4cxx11abiFALSE-cp310-cp310-linux_x86_64.whl

Processing ./flash_attn-2.6.3+cu123torch2.4cxx11abiFALSE-cp310-cp310-linux_x86_64.whl


In [2]:
# pip install transformers==4.37.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 70.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 77.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
Note: you may need to restart the kernel to use updated packages.


# Build

## Config

In [1]:
import os
import numpy as np
import torch
import torchvision.transforms as T
# from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

import matplotlib.pyplot as plt
import glob

In [2]:
model_name = "5CD-AI/Vintern-1B-v3_5"

In [3]:
try:
  model = AutoModel.from_pretrained(
      model_name,
      torch_dtype=torch.bfloat16,
      low_cpu_mem_usage=True,
      trust_remote_code=True,
      use_flash_attn=False,
  ).eval().cuda()
except:
  model = AutoModel.from_pretrained(
      model_name,
      torch_dtype=torch.bfloat16,
      low_cpu_mem_usage=True,
      trust_remote_code=True
  ).eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)

config.json:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/3.88k [00:00<?, ?B/s]

configuration_intern_vit.py:   0%|          | 0.00/5.55k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/5CD-AI/Vintern-1B-v3_5:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/5CD-AI/Vintern-1B-v3_5:
- configuration_internvl_chat.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_internvl_chat.py:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/9.02k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.38M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

In [4]:
generation_config = dict(
    max_new_tokens=512,
    do_sample=False,
    num_beams=3,
    
    repetition_penalty=2.5,
    pad_token_id=tokenizer.eos_token_id # Add this line to gen open-ended
)

In [10]:
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

# Inferences

## Load image

In [8]:
# def load_image_from_subfolders(folder_path):
#   image_files = []
#   for root, _, files in os.walk(folder_path):  # os.walk iterates through subfolders
#       for filename in files:

#         if filename.endswith(('.jpg', '.jpeg', '.png')):
#           image_files.append(os.path.join(root, filename))
#   return image_files


In [11]:
def load_image_from_folder(folder_path):
  image_files = []
  for filename in os.listdir(folder_path):
      if filename.endswith(('.jpg', '.jpeg', '.png')):
          image_files.append(os.path.join(folder_path, filename))
  return image_files


In [10]:
# folder_path = '/kaggle/input/openvivqa'

In [14]:
folder_path = '/kaggle/input/evjvqa/train-images/train-images'

In [12]:
# image_paths = load_image_from_subfolders(folder_path)
# image_paths.sort()
# image_paths

In [15]:
image_paths = load_image_from_folder(folder_path)
image_paths.sort()
image_paths

['/kaggle/input/evjvqa/train-images/train-images/00000000002.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000003.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000004.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000005.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000006.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000009.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000010.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000017.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000020.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000022.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000023.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000025.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000026.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000031.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000000032.j

In [16]:
len(image_paths)

3767

In [ ]:
# import os

# def convert_name(image_path):
#   filename = os.path.basename(image_path)
#   name, ext = os.path.splitext(filename)
#   return name

In [15]:
# def convert_ID(path):
#   ID = int(path.split('/')[-1].split('.')[0]) #Use for EVJVQA
#   # ID = int(os.path.basename(path).split('_')[1].split('.')[0]) #Use for ViVQA4Edu
#   return ID

# Extract Ques + Ans

In [17]:
# # Extract Ques
# import pandas as pd
# qa_id_pd = pd.read_csv('/kaggle/input/evjvqa/evjvqa_annotations.csv')
# qa_id_pd

,id,image_id,question,answer
0,0,2301,what color is the shirt of the girl wearing gl...,the girl wearing glasses wears a red shirt
1,1,2301,what fruit is put on the table next to the gir...,the girl in red t-shirt is standing next to th...
2,2,2301,what are put on the table next to the woman in...,bunches of lychee
3,3,2302,what is this woman looking at?,she is looking at the milk
4,4,2302,what is the woman in white shirt doing?,the woman in white shirt is holding and lookin...
...,...,...,...,...
23780,23780,1781,この女の子は顔に何を着ていますか?,女の子はサングラスをかけています
23781,23781,1786,デンタルセンターの前に駐車している車は何ですか?,車です
23782,23782,1790,隣に立っている青い服の女性は誰ですか?,青い服の女性の隣には赤いズボンの男です
23783,23783,1790,赤いズボンの男は何を見ていますか?,赤いズボンの男は展示物を見ているます


In [18]:
# # Create a dictionary to store the merged data
# merged_data = {}

# # Iterate through the DataFrame
# for index, row in qa_id_pd[7204:15524].iterrows():
#     image_id = row['image_id']
#     question = row['question']
#     answer = row['answer']

#     if image_id not in merged_data:
#         merged_data[image_id] = {'Ques': [], 'Ans': []}

#     merged_data[image_id]['Ques'].append(question)
#     merged_data[image_id]['Ans'].append(answer)

# # Create lists to store the final output
# image_ids = []
# questions_list = []
# answers_list = []

# # Iterate through the merged data and create the final lists
# for image_id, data in merged_data.items():
#   image_ids.append(image_id)
#   questions_list.append(data['Ques'])
#   answers_list.append(data['Ans'])

# # Create a new DataFrame from the lists
# new_df = pd.DataFrame({'ImageID': image_ids, 'Ques': questions_list, 'Ans': answers_list})
# new_df['Num_Ques'] = new_df['Ques'].apply(len)
# new_df['Num_Ans'] = new_df['Ans'].apply(len)


# # Print or save the new DataFrame as needed
# new_df
# # new_df.to_csv("merged_data.csv", index=False)

,ImageID,Ques,Ans,Num_Ques,Num_Ans
0,2301,"[cô gái đeo kính mặc áo màu gì?, cô gái mặc áo...","[cô gái đeo kính mặc áo màu đỏ, cô gái mặc áo ...",3,3
1,2302,"[người phụ nữ này đang coi cái gì?, người phụ ...","[cô ấy đang coi hộp sữa, người phụ nữ áo trắng...",3,3
2,2304,"[cô gái áo hồng có đeo cái gì ngang bụng?, hai...","[cô gái áo hồng đeo túi xách màu đen, hai ngườ...",3,3
3,2308,[cô gái mặc áo khoác xanh đang đến quầy hàng n...,"[cô ấy đến để mua đồ trang trí giáng sinh, cùn...",4,4
4,2310,"[cô gái đang giơ điện thoại ra để làm gì?, cô ...","[cô gái đang giơ điện thoại ra để quét mã qr, ...",2,2
...,...,...,...,...,...
3703,1773,"[người đàn ông ngồi bên micro đang mặc đồ gì?,...","[ông ấy mặc bộ áo vest màu đen với cà vạt tím,...",3,3
3704,1777,"[có những ai được trao những bó hoa?, có những...","[ba người đứng ở giữa, hai người đàn ông bên n...",2,2
3705,1781,"[cô gái đang cầm trên tay cái gì?, cô gái đặt ...","[cô gái cầm quyển sách, chiếc túi xách, cô gái...",3,3
3706,1786,[có những loại phương tiện nào được đỗ trước t...,[những chiếc xe hơi],1,1


In [19]:
# original_question_list = new_df['Ques'].tolist()
# original_answer_list = new_df['Ans'].tolist()
# image_id = new_df['ImageID'].tolist()

# original_question_list[0]

['cô gái đeo kính mặc áo màu gì?',
 'cô gái mặc áo đỏ đang đứng cạnh bàn trưng bày loại nông sản gì?',
 'người ta trưng bày những gì trên chiếc bàn cạnh cô gái áo đỏ?']

In [20]:
# def convert_to_numbered_questions(question_list):
#   numbered_questions = ""
#   for i, question in enumerate(question_list):
#       numbered_questions += f"Câu hỏi {i+1}: {question}\n"
#   return numbered_questions[:-1] # Remove the last newline character

In [21]:
# question_list = [convert_to_numbered_questions(questions) for questions in original_question_list]
# answer_list = [convert_to_numbered_questions(questions) for questions in original_answer_list]

In [22]:
# import pandas as pd

# image_id_pd = pd.read_csv('/kaggle/input/evjvqa/evjvqa_images.csv')
# image_id_pd

,id,filename
0,2301,00000002301.jpg
1,2302,00000002302.jpg
2,2304,00000002304.jpg
3,2308,00000002308.jpg
4,2310,00000002310.jpg
...,...,...
3758,1735,00000001735.jpg
3759,1786,00000001786.jpg
3760,2441,00000002441.jpg
3761,5152,00000005152.jpg


In [23]:
# def find_filenames_by_id(df, id_list):
#   filenames = []
#   for image_id in id_list:
#     matching_rows = df[df['id'] == image_id]
#     if not matching_rows.empty:
#       filenames.extend(matching_rows['filename'].tolist())
#   return filenames

In [24]:
# filenames = find_filenames_by_id(image_id_pd, image_id)
# image_paths = [f'/kaggle/input/evjvqa/train-images/train-images/{filename}' for filename in filenames]
# image_paths

['/kaggle/input/evjvqa/train-images/train-images/00000002301.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002302.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002304.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002308.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002310.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002311.png',
 '/kaggle/input/evjvqa/train-images/train-images/00000002314.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002318.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002319.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002328.jpeg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002333.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002338.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002339.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002340.jpg',
 '/kaggle/input/evjvqa/train-images/train-images/00000002345.

## Visual Question Answering

In [17]:
import pandas as pd

question_df = pd.read_csv("/kaggle/input/evjvqa-question/Question_merged_file.csv")
question_df

,ImageID,Question,Num_Ques
0,2,['Câu hỏi 1: Những người trên thuyền đang làm ...,5
1,3,['Câu hỏi 1: Người phụ nữ trong ảnh đang làm g...,5
2,4,['Câu hỏi 1: Có bao nhiêu lồng đèn trong ảnh?'...,5
3,5,"['Câu hỏi 1: Bức ảnh này được chụp ở đâu?', 'C...",5
4,6,['Câu hỏi 1: Món ăn nào đang được phục vụ tại ...,5
...,...,...,...
3762,8680,['Câu hỏi 1: Bức tranh tường phía trên mô tả n...,4
3763,9575,['Câu hỏi 1: Người đàn ông trong ảnh đang cầm ...,5
3764,11054,['Câu hỏi 1: Hai người trong ảnh đang mặc tran...,5
3765,2237,['Câu hỏi 1: Có bao nhiêu xe máy đang đỗ trước...,5


In [18]:
root_folder = '/kaggle/input/evjvqa/train-images/train-images'

In [19]:
evjvqa_df = pd.read_csv('/kaggle/input/evjvqa/evjvqa_images.csv')
evjvqa_df

,id,filename
0,2301,00000002301.jpg
1,2302,00000002302.jpg
2,2304,00000002304.jpg
3,2308,00000002308.jpg
4,2310,00000002310.jpg
...,...,...
3758,1735,00000001735.jpg
3759,1786,00000001786.jpg
3760,2441,00000002441.jpg
3761,5152,00000005152.jpg


In [20]:
def convert_EVJVQA(file_name, root_folder):
  return os.path.join(root_folder, file_name)

def convert_ViVQA4Edu(file_name, root_folder):
  subfolder_name = file_name.split('_')[0]
  return os.path.join(root_folder, subfolder_name, file_name)

In [21]:
question_collection_list = question_df['Question'].tolist()
question_collection_list[0]

"['Câu hỏi 1: Những người trên thuyền đang làm gì?', 'Câu hỏi 2: Cánh đồng trong ảnh có màu gì?', 'Câu hỏi 3: Có bao nhiêu chiếc thuyền được nhìn thấy trong ảnh?', 'Câu hỏi 4: Loại cây gì được trồng trên cánh đồng?', 'Câu hỏi 5: Thời tiết trong ảnh như thế nào?']"

In [22]:
num_list = question_df['Num_Ques'].tolist()
num_list[0]

5

In [23]:
import ast
import re

def format_question(question):
  return re.sub(r"^Câu hỏi \d+:\s*", "", question)

In [ ]:
answers_result_list = []
imageID_list = []
num_ans_list = []

start = 1000
end = 2000

for index in range(start, end):
# for index in range(2):
    ID = question_df['ImageID'][index]
    print(ID)
    try:
        file_name = evjvqa_df[evjvqa_df['id'] == ID]['filename'].iloc[0]
        print(file_name)
        # file_name = ID + '.png'
      # full_path = convert_ViVQA4Edu(file_name, root_folder)
        full_path = convert_EVJVQA(file_name, root_folder)
    except:
        print(f"File not found: {ID}")
        answer_list.append("Lỗi không có ảnh")
        continue
        
    answer_list = []
    if full_path:
        num_ans = num_list[index]
        pixel_values = load_image(full_path, max_num=6).to(torch.bfloat16).cuda()
        image_ques_list = ast.literal_eval(question_collection_list[index])
        for ques_idx, question in enumerate(image_ques_list, 1):
            question = format_question(question)
            print(question)
            response = model.chat(tokenizer, pixel_values, question, generation_config)
            # answer = f"Câu trả lời cho câu hỏi {ques_idx}: {response}"
            answer = response
            answer_list.append(answer)
            # print(f'User: {question}\nAssistant: {response}')
        imageID_list.append(ID)
        answers_result_list.append(answer_list)
        num_ans_list.append(num_ans)
    else:
        print("Image not found")

2864
00000002864.jpg
Tòa nhà trong ảnh có kiến trúc như thế nào?
Màu sắc chủ đạo của tòa nhà là gì?
Dòng nước trước tòa nhà có màu gì?
Nội dung trên cùng của ảnh viết bằng ngôn ngữ gì?
Nội dung "Cố đô Huế - Lịch sử và Văn hóa" được viết bằng những ngôn ngữ nào khác?
2865
00000002865.jpg
Dòng chữ trên cổng chào có nội dung gì?
Tên của di tích văn hóa lịch sử được nhắc đến trong ảnh là gì?
Phương tiện giao thông nào xuất hiện trong ảnh?
2870
00000002870.jpg
Có bao nhiêu người đang đứng trên vỉa hè?
Có bao nhiêu chiếc ghế dài được đặt dọc theo bức tường?
Những bức tranh tường trên tường thể hiện những chủ đề gì?
Có bao nhiêu cây xanh được nhìn thấy trong ảnh?
Màu sắc chủ đạo của trang phục mà người phụ nữ đang mặc là gì?
2879
00000002879.jpg
Sự kiện gì đang diễn ra trong ảnh?
Các em bé trong ảnh đang mặc trang phục gì?
Các mặt hàng nào được bày bán trong gian hàng?
Gian hàng được trang trí bằng những vật liệu gì?
Có bao nhiêu người đang đeo khẩu trang trong ảnh?
2882
00000002882.png
Hai n

In [31]:
# import ast
# import re

# # def format_question(question):
# #   return re.sub(r"^Câu hỏi \d+:\s*", "", question)


# start = 2500
# end = 3708

# answers_result_list = []
# imageID_list = []
# num_ans_list = []

# # for index in range(len(path_list[start:end])):
# # for index in range(2):
# for index in range(start,end):
#     # image_path = find_image(path_list[index], folder_path)
#     image_path = image_paths[index]
#     answer_list = []
#     if image_path:
#         # num_ans = num_list[index]
#         num_ans = new_df['Num_Ans'][index]
#         # imageID = convert_ID(image_path)
#         imageID = image_id[index]
#         pixel_values = load_image(image_path, max_num=6).to(torch.bfloat16).cuda()
#         # image_ques_list = ast.literal_eval(question_collection_list[index])
#         image_ques_list = original_question_list[index]
#         for ques_idx, question in enumerate(image_ques_list, 1):
#             # question = format_question(question)
#             response = model.chat(tokenizer, pixel_values, question, generation_config)
#             answer = f"Câu trả lời cho câu hỏi {ques_idx}: {response}"
#             answer_list.append(answer)
#             # print(f'User: {question}\nAssistant: {response}')
#         print(imageID)
#         imageID_list.append(imageID)
#         answers_result_list.append(answer_list)
#         num_ans_list.append(num_ans)
#     else:
#         print("Image not found")

9520
9521
9522
9523
9524
9525
9527
9528
9529
9530
9531
9532
9535
9536
9540
9541
9546
9547
9548
9549
9550
9552
9555
9558
9559
9560
9561
9564
9566
9567
9568
9569
9570
9571
9572
9573
9574
9575
9576
9578
9579
9580
9583
9584
9587
9588
9589
9593
9594
9598
3002
3004
3005
3008
3012
3015
3016
3017
3021
3024
3025
3028
3035
3037
3041
3042
3044
3045
3049
3050
3053
3054
3055
3057
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3074
3076
3077
3078
3079
3083
3084
3085
3086
3087
3092
3093
3095
3098
1901
1903
1904
1909
1910
1912
1922
1923
1927
1928
1931
1939
1943
1945
1946
1947
1958
1959
1960
1963
1964
1965
1967
1970
1971
1977
1988
1991
1993
1999
2602
2606
2607
2611
2614
2616
2617
2621
2623
2625
2632
2636
2637
2645
2647
2650
2653
2655
2656
2658
2660
2663
2672
2675
2676
2681
2686
2689
2697
2699
508
511
513
515
516
517
518
519
521
523
525
527
531
533
539
540
541
542
543
549
550
551
559
562
571
573
574
575
580
581
584
585
593
595
599
10300
10301
10302
10303
10307
10308
10309
10311
10312
10313
10315
1031

In [32]:
print(len(answers_result_list))
print(len(imageID_list))
print(len(num_ans_list))

1208
1208
1208


In [33]:
import csv

with open('/kaggle/working/Vintern1B_EVJVQA_Ans_2.csv', 'w', newline='', encoding='utf-8') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['ImageID', 'Answer', 'Num_Ans'])  # Write header row
  for i in range(len(imageID_list)):
    writer.writerow([imageID_list[i], answers_result_list[i], num_ans_list[i]])


In [34]:
# import os

# # Specify the path to the file you want to remove
# file_path = "/kaggle/working/flash_attn-2.6.3+cu123torch2.4cxx11abiFALSE-cp310-cp310-linux_x86_64.whl.2"

# # Check if the file exists, then remove it
# if os.path.exists(file_path):
#     os.remove(file_path)
#     print(f"File '{file_path}' has been removed.")
# else:
#     print(f"File '{file_path}' does not exist.")